In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

d:\projects\sentiment_analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("sismetanin/rubert-ru-sentiment-rusentiment")
model = AutoModelForSequenceClassification.from_pretrained("sismetanin/rubert-ru-sentiment-rusentiment")




In [25]:
inputs = tokenizer("Очень плохие наушники", padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(torch.nn.functional.softmax(logits))

tensor([[0.1847, 0.7454, 0.0160, 0.0526, 0.0013]])


C:\Users\пк\AppData\Local\Temp\ipykernel_15136\4150916011.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.nn.functional.softmax(logits))


In [26]:
texts = [
    "Этот фильм был просто ужасен!",  # ожидаем негатив
    "Фильм нормальный, не хуже других.",  # ожидаем нейтральный
    "Мне очень понравилось! Это шедевр!",  # ожидаем очень позитивный
]

for text in texts:
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    
    print(f"Текст: {text}")
    print(f"Логиты: {logits}")
    print(f"Предсказанный класс: {predicted_class} ({['Neutral', 'Positive', 'Very Positive', 'Negative', 'Very Negative'][predicted_class]})")
    print("-" * 50)

Текст: Этот фильм был просто ужасен!
Логиты: tensor([[ 2.8753,  0.2771,  0.2848,  0.5628, -3.2889]])
Предсказанный класс: 0 (Neutral)
--------------------------------------------------
Текст: Фильм нормальный, не хуже других.
Логиты: tensor([[-0.5001,  0.2734,  3.5308, -0.1880, -3.1131]])
Предсказанный класс: 2 (Very Positive)
--------------------------------------------------
Текст: Мне очень понравилось! Это шедевр!
Логиты: tensor([[-0.9837, -1.5925,  4.6931, -1.1424, -1.2629]])
Предсказанный класс: 2 (Very Positive)
--------------------------------------------------


In [2]:
import pandas as pd

df = pd.read_csv("reviews.csv", sep="\t")
print(df.head())


                                              review sentiment
0  качество плохое пошив ужасный (горловина напер...  negative
1  Товар отдали другому человеку, я не получила п...  negative
2  Ужасная синтетика! Тонкая, ничего общего с пре...  negative
3  товар не пришел, продавец продлил защиту без м...  negative
4      Кофточка голая синтетика, носить не возможно.  negative


In [6]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
from transformers import DataCollatorWithPadding

# Класс датасета
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep="\t")
    label_mapping = {"negative": 0, "neautral": 1, "positive": 2}
    df["sentiment"] = df["sentiment"].map(label_mapping)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Перемешивание
    df = df.head(2000)
    train_texts, test_texts, train_labels, test_labels = train_test_split(
        df['review'], df['sentiment'], test_size=0.2, random_state=42
    )
    return SentimentDataset(train_texts.tolist(), train_labels.tolist()), SentimentDataset(test_texts.tolist(), test_labels.tolist())


class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        return {"input_ids": encoding["input_ids"].squeeze(0),
                "attention_mask": encoding["attention_mask"].squeeze(0),
                "label": torch.tensor(self.labels[idx], dtype=torch.long)}

# Выбор модели
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 класса (negative, neutral, positive)

# Подготовка датасета
train_dataset, test_dataset = load_dataset("reviews.csv")
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Определение метрик
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    
    return {"accuracy": acc, "f1": f1, "recall": recall}

# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results2",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs2",  
    logging_strategy="steps",
    logging_steps=50,
    report_to="tensorboard" 
)



data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,  # Замена устаревшего tokenizer
    compute_metrics=compute_metrics,
)

trainer.train()

# Сохранение модели
model.save_pretrained("./finetuned_rubert_tiny2")
tokenizer.save_pretrained("./finetuned_rubert_tiny2")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall
1,0.909500,0.869656,0.645000,0.645532,0.645000


('./finetuned_rubert_tiny2\\tokenizer_config.json',
 './finetuned_rubert_tiny2\\special_tokens_map.json',
 './finetuned_rubert_tiny2\\vocab.txt',
 './finetuned_rubert_tiny2\\added_tokens.json',
 './finetuned_rubert_tiny2\\tokenizer.json')

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import time

# Загрузка сохраненной модели и токенайзера
model_path = "./finetuned_rubert_tiny"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()  
# Маппинг числовых меток в текстовые
label_mapping = {0: "negative", 1: "neutral", 2: "positive"}

# Функция для предсказания сентимента
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return label_mapping[predicted_class]

start = time.time()
# Пример использования
texts = [
    "Мне очень понравилась эта книга",
]

for text in texts:
    sentiment = predict_sentiment(text)
    out = f"Текст: {text}\nСентимент: {sentiment}\n"
end = time.time()
print(end - start)
print(out)

0.05047130584716797
Текст: Мне очень понравилась эта книга
Сентимент: positive



In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score, recall_score, precision_score
import pandas as pd
import time

# Загрузка сохраненной модели и токенайзера
model_path = "./finetuned_rubert"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

# Маппинг меток
label_mapping = {0: "B", 1: "N", 2: "G"}
reverse_label_mapping = {"B": 0, "N": 1, "G": 2}

# Функция для предсказания сентимента
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return label_mapping[predicted_class]

# Открытие датасета
dataset_path = "dataset_comments_35.xlsx"
df = pd.read_excel(dataset_path)

true_labels = []
predicted_labels = []

start = time.time()

for index, row in df.iterrows():
    text = row['MessageText']
    true_label = row['Class']
    predicted_label = predict_sentiment(text)
    
    true_labels.append(reverse_label_mapping[true_label])
    predicted_labels.append(reverse_label_mapping[predicted_label])

end = time.time()
print(f"Время выполнения: {end - start} секунд")

# Расчет метрик
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Время выполнения: 12.094168424606323 секунд
Precision: 0.6500
Recall: 0.6535
F1 Score: 0.6492


In [ ]:
import torch
import onnx

model_path = "./finetuned_rubert"
onnx_model_path = "./finetuned_rubert.onnx"

# Загружаем PyTorch-модель
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

# Пример входных данных
dummy_input = torch.randint(0, 30522, (1, 512))  # Входной тензор аналогичный токенизированному тексту

# Экспорт модели в ONNX с opset_version=14
torch.onnx.export(
    model, 
    dummy_input, 
    onnx_model_path,
    input_names=["input_ids"], 
    output_names=["logits"],
    dynamic_axes={"input_ids": {0: "batch_size"}, "logits": {0: "batch_size"}},
    opset_version=14  # Устанавливаем версию 14
)

print(f"Модель сохранена в {onnx_model_path}")


KeyboardInterrupt: 

In [11]:
import onnxruntime
import numpy as np
from transformers import AutoTokenizer
import time

label_mapping = {0: "negative", 1: "neutral", 2: "positive"}

# Загружаем ONNX-модель
onnx_model_path = "./finetuned_rubert.onnx"
session = onnxruntime.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

# Загружаем токенайзер
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Функция предсказания сентимента с ONNX
def predict_sentiment_onnx(text):
    inputs = tokenizer(text, return_tensors="np", padding="max_length", truncation=True, max_length=512)
    input_ids = inputs["input_ids"].astype(np.int64)

    # Прогон через ONNX-модель
    outputs = session.run(["logits"], {"input_ids": input_ids})
    logits = outputs[0]
    predicted_class = np.argmax(logits, axis=1)[0]

    return label_mapping[predicted_class]

# Тестируем скорость
start = time.time()
texts = ["Решил купить клавиатуру logitech k120, в итоге не возможно играть в определенные игры, к примеру fifa, где нужно бежать вверх и вперед и пробить затем по воротом на в(d), теннис и прочие где нужно зажимать стрелки и кнопку. После выявления данного бага пришёл в магазин dns, на что мне сказали, что другие проверить клавиатуры нельзя! И вообще типо так работают все мембранные клавиатуры. А дома старинная замызганная клавиатура 2005 года работает у меня успешно неизвестного бренда! Как назло выкинул коробку от этого, теперь страдаю. Не рекомендую данную клавиатуру к покупке."]

for text in texts:
    sentiment = predict_sentiment_onnx(text)
    out = f"Текст: {text}\nСентимент: {sentiment}\n"
end = time.time()

print(end - start)
print(out)


0.3879554271697998
Текст: Решил купить клавиатуру logitech k120, в итоге не возможно играть в определенные игры, к примеру fifa, где нужно бежать вверх и вперед и пробить затем по воротом на в(d), теннис и прочие где нужно зажимать стрелки и кнопку. После выявления данного бага пришёл в магазин dns, на что мне сказали, что другие проверить клавиатуры нельзя! И вообще типо так работают все мембранные клавиатуры. А дома старинная замызганная клавиатура 2005 года работает у меня успешно неизвестного бренда! Как назло выкинул коробку от этого, теперь страдаю. Не рекомендую данную клавиатуру к покупке.
Сентимент: negative



In [ ]:
import pandas as pd

splits = {'train': 'train.csv', 'validation': 'valid.csv'}
df = pd.read_csv("hf://datasets/MonoHime/ru_sentiment_dataset/" + splits["train"])


In [6]:
len(df)

189891

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
from transformers import DataCollatorWithPadding


splits = {'train': 'train.csv', 'validation': 'valid.csv'}
# Класс датасета
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep="\t")
    label_mapping = {2: 0, 0: 1, 1: 2}
    df["sentiment"] = df["sentiment"].map(label_mapping)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Перемешивание
    train_texts, test_texts, train_labels, test_labels = train_test_split(
        df['text'], df['sentiment'], test_size=0.2, random_state=42
    )
    return SentimentDataset(train_texts.tolist(), train_labels.tolist()), SentimentDataset(test_texts.tolist(), test_labels.tolist())


class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        return {"input_ids": encoding["input_ids"].squeeze(0),
                "attention_mask": encoding["attention_mask"].squeeze(0),
                "label": torch.tensor(self.labels[idx], dtype=torch.long)}

# Выбор модели
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 класса (negative, neutral, positive)

# Подготовка датасета
train_dataset, test_dataset = load_dataset("hf://datasets/MonoHime/ru_sentiment_dataset/" + splits["train"])
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Определение метрик
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    
    return {"accuracy": acc, "f1": f1, "recall": recall}

# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results_rusent",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs_rusent",  
    logging_strategy="steps",
    logging_steps=50,
    report_to="tensorboard" 
)



data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,  # Замена устаревшего tokenizer
    compute_metrics=compute_metrics,
)

trainer.train()

# Сохранение модели
model.save_pretrained("./finetuned_rubert_tiny_rusent")
tokenizer.save_pretrained("./finetuned_rubert_tiny_rusent")